# Download the data (run only once)

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nadyinky/sephora-products-and-skincare-reviews")

print("Path to dataset files:", path)

c:\Users\perwj\anaconda3\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 147M/147M [01:29<00:00, 1.71MB/s] 

Extracting files...


Path to dataset files: C:\Users\perwj\.cache\kagglehub\datasets\nadyinky\sephora-products-and-skincare-reviews\versions\2


# Load and clean the dataset

In [5]:
import pandas as pd

# Load the CSV data into a DataFrame
df = pd.read_csv("data/product_info.csv")

primary_category = set(df['primary_category'])
print(primary_category)

# Filter the DataFrame for rows where 'primary_category' is 'Skincare'
skincare_df = df[df['primary_category'] == 'Skincare']

# Display data size
print("Skin care data size", len(skincare_df))

# remove rows where highlight are non existent
skincare_df = skincare_df[skincare_df['highlights'].notna() & (skincare_df['highlights'] != '')]

print("Skin care data size after removing empty highlights",len(skincare_df))

# remove rows where ingredients are non existent
skincare_df = skincare_df[skincare_df['ingredients'].notna() & (skincare_df['ingredients'] != '')]

print("Skin care data size after removing empty ingredients",len(skincare_df))

# remove where the secondary_category is empty
skincare_df = skincare_df[skincare_df['secondary_category'].notna() & (skincare_df['secondary_category'] != '')]

print("Skin care data size after removing empty secondary_category",len(skincare_df))

# # remove where the tertiary_category is empty
# skincare_df = skincare_df[skincare_df['tertiary_category'].notna() & (skincare_df['tertiary_category'] != '')]
# print("Skin care data size after removing empty tertiary_category",len(skincare_df))


{'Makeup', 'Skincare', 'Fragrance', 'Mini Size', 'Hair', 'Bath & Body', 'Men', 'Gifts', 'Tools & Brushes'}
Skin care data size 2420
Skin care data size after removing empty highlights 2003
Skin care data size after removing empty ingredients 1927
Skin care data size after removing empty secondary_category 1927


# Clean highlights column

In [6]:
highlights = skincare_df['highlights']

highlights = [h.replace("[", "").replace("]", "").replace("'", "").replace("Best for ", "").replace("Good for: ", "").replace(" Skin", "").replace("/", ", ") for h in highlights]

skincare_df['highlights'] = highlights

# Save cleaned dataset

In [7]:
skincare_df.to_csv('data/skincare.csv', index=False)

In [12]:
# Load the CSV data into a DataFrame
df = pd.read_csv("data/reviews_0-250.csv")

product_ids = skincare_df['product_id']
print(len(df))

df_review1 = df[df['product_id'].isin(product_ids)]
print(len(df_review1))
print("Different users, ", len(set(df['author_id'])))

C:\Users\perwj\AppData\Local\Temp\ipykernel_17188\2568968107.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/reviews_0-250.csv")


602130
545756
Different users,  383697


In [13]:
df_2 = pd.read_csv("data/reviews_250-500.csv")
df_review2 = df_2[df_2['product_id'].isin(product_ids)]

df_3 = pd.read_csv("data/reviews_500-750.csv")
df_review3 = df_3[df_3['product_id'].isin(product_ids)]

df_4 = pd.read_csv("data/reviews_750-1250.csv")
df_review4 = df_4[df_4['product_id'].isin(product_ids)]

df_5 = pd.read_csv("data/reviews_1250-end.csv")
df_review5 = df_5[df_5['product_id'].isin(product_ids)]

combined_df = pd.concat([df_review2, df_review1], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review3], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review4], axis=0, ignore_index=True)
combined_df = pd.concat([combined_df, df_review5], axis=0, ignore_index=True)

print(len(combined_df))
print(len(set(combined_df['author_id'])))


C:\Users\perwj\AppData\Local\Temp\ipykernel_17188\1564603506.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_4 = pd.read_csv("data/reviews_750-1250.csv")
C:\Users\perwj\AppData\Local\Temp\ipykernel_17188\1564603506.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_5 = pd.read_csv("data/reviews_1250-end.csv")


966018
513998


### Users with at least 7 reviews

In [72]:
mini_in_product_name = combined_df['product_name'].str.contains('mini', case=False, na=False)
limited_edition_in_product_name = combined_df['product_name'].str.contains('limited edition', case=False, na=False)

rows_to_exclude = mini_in_product_name | limited_edition_in_product_name


cleaned_df = combined_df[~rows_to_exclude]


In [73]:
xxfiltered_df = combined_df.groupby('author_id').filter(lambda x: x['product_id'].nunique() > 6)
print(len(xxfiltered_df))
print(len(set(xxfiltered_df['author_id'])))

170851
13033


In [74]:
filtered_df = cleaned_df.groupby('author_id').filter(lambda x: x['product_id'].nunique() > 6)

# Display the filtered DataFrame
print(len(filtered_df))
print(len(set(filtered_df['author_id'])))
#set(filtered_df['author_id'])

126027
9259


In [75]:
selected_columns = filtered_df[['author_id', 'product_id']]
combined_reviews = selected_columns.groupby('author_id')['product_id'].apply(lambda x: ' '.join(x)).reset_index()
combined_reviews

,author_id,product_id
0,11777122,P232915 P466123 P421277 P500777 P475124 P48232...
1,634369095,P466123 P456218 P453928 P472810 P465741 P45891...
2,912025723,P399623 P500777 P480461 P504045 P475124 P47281...
3,958443488,P466123 P456213 P481697 P480461 P482008 P48174...
4,967124371,P7880 P430337 P4016 P466123 P421277 P405096 P4...
...,...,...
9254,998853649,P456211 P470041 P447212 P447782 P447781 P45408...
9255,9990263118,P456566 P467655 P440307 P410400 P432668 P43454...
9256,9994328735,P281835 P421275 P428250 P454018 P469482 P41429...
9257,9996449006,P188307 P4010 P420699 P297524 P54509 P434546 P...


In [76]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# Step 1: Convert the 'reviews' into a list of transactions
# Assuming `combined_reviews` is the DataFrame with aggregated reviews per author
transactions = combined_reviews['product_id'].str.split().tolist()

# Step 2: Create a DataFrame for one-hot encoding
# Flatten all unique items (reviews) and create a unique item list
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# Step 3: Apply the Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

# Step 4: Focus on pairs of reviews
frequent_pairs = frequent_itemsets[frequent_itemsets['itemsets'].apply(len) == 2]

# Step 5: Generate association rules (optional, for insights)
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.05, num_itemsets=len(frequent_itemsets))

# Display the frequent pairs
print(frequent_pairs)

     support            itemsets
36  0.057458  (P270594, P476414)
37  0.051085  (P270594, P479841)
38  0.079274  (P270594, P500633)
39  0.064370  (P270594, P503936)
40  0.056918  (P500633, P423688)
41  0.054434  (P479841, P476414)
42  0.058970  (P500633, P476414)
43  0.055082  (P500633, P479841)
44  0.067286  (P503936, P500633)


In [77]:
# Select and rename key columns for readability
rules_display = rules[["antecedents", "consequents", "support", "confidence", "lift"]].copy()

# Convert frozensets to readable strings
rules_display["antecedents"] = rules_display["antecedents"].apply(lambda x: ', '.join(list(x)))
rules_display["consequents"] = rules_display["consequents"].apply(lambda x: ', '.join(list(x)))

# Sort by confidence or any other metric (optional)
rules_display = rules_display.sort_values(by="confidence", ascending=False)

# Print the rules as a nice table
print(rules_display.to_string(index=False))


antecedents consequents  support  confidence     lift
    P503936     P500633 0.067286    0.575254 3.668234
    P479841     P500633 0.055082    0.564159 3.597487
    P479841     P476414 0.054434    0.557522 4.629684
    P503936     P270594 0.064370    0.550323 3.671068
    P270594     P500633 0.079274    0.528818 3.372128
    P479841     P270594 0.051085    0.523230 3.490337
    P500633     P270594 0.079274    0.505510 3.372128
    P476414     P500633 0.058970    0.489686 3.122592
    P476414     P270594 0.057458    0.477130 3.182815
    P476414     P479841 0.054434    0.452018 4.629684
    P270594     P503936 0.064370    0.429395 3.671068
    P500633     P503936 0.067286    0.429063 3.668234
    P270594     P476414 0.057458    0.383285 3.182815
    P500633     P476414 0.058970    0.376033 3.122592
    P500633     P423688 0.056918    0.362948 2.126919
    P500633     P479841 0.055082    0.351240 3.597487
    P270594     P479841 0.051085    0.340778 3.490337
    P423688     P500633 0.05

In [ ]:
# let's try to quantify this
